# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [25]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path

# Import API key
from api_keys import geoapify_key

In [26]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("output_data/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,hermanus,-34.4187,19.2345,8.94,74,49,3.20,ZA,1690869377
1,1,nova vicosa,-17.8919,-39.3719,20.96,76,69,2.36,BR,1690869377
2,2,mascota,20.5167,-104.7833,19.40,95,100,0.32,MX,1690869378
3,3,saipan,15.1355,145.7010,30.39,100,75,0.45,MP,1690869378
4,4,saint-pol-de-leon,48.6833,-3.9833,16.55,89,83,3.58,FR,1690869379


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [27]:
%%capture --no-display

# Configure the map plot
city_map = city_data_df.hvplot.points("Lng", 
                                      "Lat", 
                                      geo = True,
                                      color = "City",
                                      alpha = 0.6,
                                      size = "Humidity",
                                      tiles = "OSM",
                                      width  =1000,
                                      height = 560
                                     )

# Display the map
city_map


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [28]:
# Narrow down cities that fit criteria and drop any results with null values

# My Criteria:
# A max temperature lower than 27 degrees but higher than 21
# Cloudiness less than 3
# Wind speed less than 4.5 m/s


weather_filtered_df = city_data_df.loc[(city_data_df["Max Temp"] < 27)
                                       & (city_data_df["Max Temp"] > 21) 
                                       & (city_data_df["Cloudiness"] < 3)
                                       & (city_data_df["Wind Speed"] < 4.5)
                                      ]

# Drop any rows with null values
weather_filtered_df.dropna()

# Display sample data
weather_filtered_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
17,17,san felice circeo,41.2372,13.0942,26.06,99,0,0.45,IT,1690869386
52,52,portland,45.5234,-122.6762,22.99,62,0,3.60,US,1690869271
57,57,elizabeth city,36.2946,-76.2511,22.01,96,0,0.00,US,1690869181
60,60,podgornoye,51.7350,39.1496,22.12,78,0,2.00,RU,1690869412
74,74,ras el aioun,35.6738,5.6453,26.12,39,0,1.57,DZ,1690869420
118,118,moree,-29.4667,149.8500,21.50,26,0,3.88,AU,1690869446
164,164,north platte,41.1239,-100.7654,22.84,93,0,3.13,US,1690869474
200,200,riberalta,-10.9833,-66.1000,21.66,57,1,0.44,BO,1690869495
206,206,giado,31.9582,12.0203,24.65,48,0,1.40,LY,1690869499
220,220,natal,-5.7950,-35.2094,22.12,100,0,3.60,BR,1690869507


### Step 3: Create a new DataFrame called `hotel_df`.

In [29]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = weather_filtered_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
17,san felice circeo,IT,41.2372,13.0942,99,
52,portland,US,45.5234,-122.6762,62,
57,elizabeth city,US,36.2946,-76.2511,96,
60,podgornoye,RU,51.7350,39.1496,78,
74,ras el aioun,DZ,35.6738,5.6453,39,
118,moree,AU,-29.4667,149.8500,26,
164,north platte,US,41.1239,-100.7654,93,
200,riberalta,BO,-10.9833,-66.1000,57,
206,giado,LY,31.9582,12.0203,48,
220,natal,BR,-5.7950,-35.2094,100,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [30]:
# Set parameters to search for a hotel
radius = 10000
params = {"categories": "accommodation.hotel",
          "apiKey":geoapify_key
         }

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitude = row["Lng"]
    latitude = row["Lat"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] =  f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params = params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search


san felice circeo - nearest hotel: Centro Ferie Salvatore
portland - nearest hotel: the Hoxton
elizabeth city - nearest hotel: Quality Inn
podgornoye - nearest hotel: Норд
ras el aioun - nearest hotel: No hotel found
moree - nearest hotel: No hotel found
north platte - nearest hotel: Howard Johnson Inn - North Platte
riberalta - nearest hotel: Hotel Jomali
giado - nearest hotel: فندق جادو السياحي
natal - nearest hotel: Natal Palace Hotel
le francois - nearest hotel: Hôtel la Frégate Bleu
anacapri - nearest hotel: No hotel found
la junta - nearest hotel: Travel Inn of La Junta
tefenni - nearest hotel: Barutlusu Tesisleri
sinop - nearest hotel: No hotel found
kharbatha al misbah - nearest hotel: מלון לוגוס
acarlar - nearest hotel: No hotel found
posto fiscal rolim de moura - nearest hotel: No hotel found
arta - nearest hotel: Cronos
khorugh - nearest hotel: кивекас
volokolamsk - nearest hotel: No hotel found
anna regina - nearest hotel: Jaigobin Hotel
malabar - nearest hotel: Coogee Bay 

,City,Country,Lat,Lng,Humidity,Hotel Name
17,san felice circeo,IT,41.2372,13.0942,99,Centro Ferie Salvatore
52,portland,US,45.5234,-122.6762,62,the Hoxton
57,elizabeth city,US,36.2946,-76.2511,96,Quality Inn
60,podgornoye,RU,51.7350,39.1496,78,Норд
74,ras el aioun,DZ,35.6738,5.6453,39,No hotel found
118,moree,AU,-29.4667,149.8500,26,No hotel found
164,north platte,US,41.1239,-100.7654,93,Howard Johnson Inn - North Platte
200,riberalta,BO,-10.9833,-66.1000,57,Hotel Jomali
206,giado,LY,31.9582,12.0203,48,فندق جادو السياحي
220,natal,BR,-5.7950,-35.2094,100,Natal Palace Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [31]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points("Lng", 
                                   "Lat", 
                                   geo = True,
                                   color = "City",
                                   alpha = 0.8,
                                   size = "Humidity",
                                   tiles = "OSM",
                                   hover_cols = ["Hotel Name", "Country"],
                                   width  =1000,
                                   height = 560
                                  )

# Display the map
hotel_map


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)